# Docling PDF Reader — Basic RAG

In [ ]:
import rich
from rich.pretty import pprint
import warnings
import os
from dotenv import load_dotenv

load_dotenv()
warnings.filterwarnings(
    action="ignore", category=FutureWarning, module="easyocr"
)
warnings.filterwarnings(
    action="ignore", category=UserWarning, module="pydantic"
)

## Basic usage

By default, `DoclingPDFReader` exports to Markdown. Basic usage looks like this:

In [ ]:
from llama_index.readers.docling import DoclingPDFReader

reader = DoclingPDFReader()
docs = reader.load_data(
    file_path="https://arxiv.org/pdf/2408.09869",  # PDF local path or URL (or iterable thereof)
)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Let's preview a doc sample:

In [ ]:
md_snippet = docs[0].text[408:1200]
rich.print(f"...{md_snippet}...")

...
## Abstract

This technical report introduces Docling , an easy to use, self-contained, MITlicensed open-source package for PDF 
document conversion. It is powered by state-of-the-art specialized AI models for layout analysis (DocLayNet) and 
table structure recognition (TableFormer), and runs efficiently on commodity hardware in a small resource budget. 
The code interface allows for easy extensibility and addition of new features and models.

## 1 Introduction

Converting PDF documents back into a machine-processable format has been a major challenge for decades due to their
huge variability in formats, weak standardization and printing-optimized characteristic, which discards most 
structural features and metadata. With the advent of LLMs and popular application patterns such as re...

## RAG demo

Next, we extract the chunks — and preview a chunk sample:

In [ ]:
from llama_index.core.node_parser import MarkdownNodeParser

node_parser = MarkdownNodeParser()
nodes = node_parser.get_nodes_from_documents(documents=docs)
pprint(nodes[7], max_length=2, max_string=250, max_depth=2)

TextNode(
│   id_='d69d4174-500b-4aae-ae56-adb9da53520e',
│   embedding=None,
│   metadata={
│   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   'Header_2': '3.1 PDF backends'
│   },
│   excluded_embed_metadata_keys=['dl_doc_hash'],
│   excluded_llm_metadata_keys=['dl_doc_hash'],
│   relationships={
│   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(...),
│   │   <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(...),
│   │   ... +1
│   },
│   text='3.1 PDF backends\n\nTwo basic requirements to process PDF documents in our pipeline are a) to retrieve all text content and their geometric coordinates on each page and b) to render the visual representation of each page as it would appear in a PDF vie'+1029,
│   mimetype='text/plain',
│   start_char_idx=5009,
│   end_char_idx=6288,
│   text_template='{metadata_str}\n\n{content}',
│   metadata_template='{key}: {value}',
│   metadata_seperator='\n'
)

We now define the embed model:

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

We set up the vector store:

In [ ]:
from tempfile import TemporaryDirectory
from llama_index.vector_stores.milvus import MilvusVectorStore

vector_store = MilvusVectorStore(
    uri=os.environ.get(
        "MILVUS_URI",
        default=f"{(tmp_dir := TemporaryDirectory()).name}/milvus_demo.db",
    ),
    collection_name="docling_collection",
    dim=len(embed_model.get_text_embedding("hi")),
    overwrite=True,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


We create the `VectorStoreIndex`, triggering the encoding of the chunks and the ingestion into the vector store:

In [ ]:
from llama_index.core import StorageContext, VectorStoreIndex

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    nodes=nodes,
    embed_model=embed_model,
    storage_context=storage_context,
    show_progress=True,
)

Generating embeddings:   0%|          | 0/33 [00:00<?, ?it/s]

Last but not least, we set up the LLM:

In [ ]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

HF_TOKEN = os.environ.get("HF_TOKEN")

llm = HuggingFaceInferenceAPI(
    token=HF_TOKEN,
    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
)

With all pieces in place, we are now ready to ask questions on our document content.

In [ ]:
query_engine = index.as_query_engine(llm=llm)
query_res = query_engine.query("Which are the main AI models in Docling?")
pprint(query_res, max_length=5, max_string=170, max_depth=4)

Response(
│   response='\nThe main AI models in Docling are a layout analysis model, which is an accurate object-detector for page elements, and TableFormer, a state-of-the-art table structure re'+16,
│   source_nodes=[
│   │   NodeWithScore(
│   │   │   node=TextNode(
│   │   │   │   id_='f8652b3b-ae70-4942-b6d6-d183d56b95cf',
│   │   │   │   embedding=None,
│   │   │   │   metadata={...},
│   │   │   │   excluded_embed_metadata_keys=[...],
│   │   │   │   excluded_llm_metadata_keys=[...],
│   │   │   │   relationships={...},
│   │   │   │   text='3.2 AI models\n\nAs part of Docling, we initially release two highly capable AI models to the open-source community, which have been developed and published recently by our'+453,
│   │   │   │   mimetype='text/plain',
│   │   │   │   start_char_idx=6293,
│   │   │   │   end_char_idx=6916,
│   │   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   │   metadata_template='{key}: {value}',
│   │   │   │   metadata_seperator='\n'
│   │   │   ),
│   │   │   score=0.8273308277130127
│   │   ),
│   │   NodeWithScore(
│   │   │   node=TextNode(
│   │   │   │   id_='b11dca38-3b42-4e59-9c42-21b1dca89158',
│   │   │   │   embedding=None,
│   │   │   │   metadata={...},
│   │   │   │   excluded_embed_metadata_keys=[...],
│   │   │   │   excluded_llm_metadata_keys=[...],
│   │   │   │   relationships={...},
│   │   │   │   text='5 Applications\n\nThanks to the high-quality, richly structured document conversion achieved by Docling, its output qualifies for numerous downstream applications. For exam'+1035,
│   │   │   │   mimetype='text/plain',
│   │   │   │   start_char_idx=14147,
│   │   │   │   end_char_idx=15352,
│   │   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   │   metadata_template='{key}: {value}',
│   │   │   │   metadata_seperator='\n'
│   │   │   ),
│   │   │   score=0.7876063585281372
│   │   )
│   ],
│   metadata={
│   │   'f8652b3b-ae70-4942-b6d6-d183d56b95cf': {
│   │   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   │   'Header_2': '3.2 AI models'
│   │   },
│   │   'b11dca38-3b42-4e59-9c42-21b1dca89158': {
│   │   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   │   'Header_2': '5 Applications'
│   │   }
│   }
)